# Análise Exploratória de Dados
## *Vulcanismo no Holoceno do "The Smithsonian Institution"*
#### Fonte: [https://www.kaggle.com/smithsonian/volcanic-eruptions](https://www.kaggle.com/smithsonian/volcanic-eruptions)



# Configuração de ambiente


## Instalação de dependências

In [0]:
# Instalação do Basemap e dependências para dados geoespaciais
!apt install proj-bin libproj-dev libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/v1.1.0.tar.gz

## Importar bibliotecas

In [0]:
import numpy as np
import pandas as pd
from pandas import Series

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [0]:
# Matplotlib style
plt.style.use('default')

## Subir arquivos
Subir o arquivo **`volcanic-eruptions-database.csv`**


In [0]:
# Subir arquivo

In [0]:
# Listar arquivos no diretório de trabalho
%ls

# Importação dos dados

In [0]:
volcanoes = pd.read_csv("volcanic-eruptions-database.csv")

# Auditando dados

Trata-se de uma série ou uma tabela de atributos?

In [0]:
volcanoes.head()
# volcanoes.tail()

Quais são os nomes (*labels*) das variáveis e dos índices?

In [0]:
volcanoes.columns

> Que tipos de objetos o pandas reconheceu durante a importação dos dados?

> Há variáveis numéricas e categóricas?

> Colunas com datas foram reconhecidas como `pandas.to_datetime` durante a importação?

In [0]:
volcanoes.info()

Resumo estatístico das variáveis numéricas:

In [0]:
volcanoes.describe()

## Variáveis categóricas

Como estão distribuídas as categorias das variáveis categóricas?

In [0]:
volcanoes['Type'].value_counts()

## Série de tempo
Conferindo os valores de tempo informados na coluna 'Last Known Eruption'.

In [0]:
# volcanoes['Last Known Eruption'].dtype
volcanoes['Last Known Eruption'].sample(5)
# volcanoes['Last Known Eruption'].replace({'Unknown': np.nan}, inplace=True)

In [0]:
# Separar os valores de tempo do período (BCE, CE)
volcanoes[['Year', 'Period']] = volcanoes['Last Known Eruption'].str.split(' ', expand=True)

In [0]:
volcanoes['Year'].sample(5)

In [0]:
volcanoes['Period'].unique()

In [0]:
# Converter valores em números
volcanoes['Year'] = pd.to_numeric(volcanoes['Year'])
volcanoes['Year'].sample(5)

In [0]:
# Converter valores de 'Year' para CE > 0, BCE < 0
mask = volcanoes['Period'] == 'BCE'
volcanoes['Year'][mask] = -volcanoes['Year']

# Visualização

## Pré-visualização

In [0]:
volcanoes.plot();

In [0]:
sns.pairplot(volcanoes, diag_kind='kde', plot_kws={'alpha': 0.2});

## Distribuição de variáveis contínuas

In [0]:
volcanoes['Elevation (Meters)'].plot();

In [0]:
volcanoes['Elevation (Meters)'].plot.hist();

In [0]:
volcanoes['Elevation (Meters)'].plot.hist();

## Variáveis categóricas

In [0]:
def bar_plot(data):
    series = pd.Series(data).value_counts().sort_index()
    return series.plot(kind='bar')


def barh_plot(data):
    series = pd.Series(data)
    return series.value_counts().sort_index().plot.barh()


def line_plot(data):
    series = pd.Series(data).value_counts().sort_index()
    return series.plot()

Qual é a distribuição anual de atividades vulcânicas neste data set? 

A variação observada é resultado da variação de atividade vulcânica no planeta ou é um efeito da amostragem?

In [0]:
plt.figure(figsize=(12, 5))

# line_plot(volcanoes["Year"])
# bar_plot(volcanoes["Year"])

plt.ylabel("Frequência")
plt.title("Atividade vulcânica");

In [0]:
plt.figure(figsize=(5, 10))

volcanoes["Region"].value_counts().sort_values().plot.barh()

plt.ylabel("Região")
plt.title("Regiões com mais vulcanismo durante o Holoceno");

Qual é a distribuição de atividades vulcânicas por país? 

A diferença encontrada é um fator geológico ou de desenvolvimento científico?

In [0]:
plt.figure(figsize=(5, 25))

volcanoes["Country"].value_counts().sort_values().plot.barh()

plt.ylabel("País")
plt.title("Atividade vulcânica por país")
plt.show()

Quais são os tipos de evidências de vulcanismo neste dataset?

In [0]:
volcanoes["Activity Evidence"].value_counts().sort_values().plot.barh()
plt.ylabel("Evidências")
plt.title("Evidência de atividade vulcânica")
plt.show()

Quais são os tipos de rochas dominantes no dataset?

In [0]:
volcanoes["Dominant Rock Type"].value_counts().sort_values().plot.barh()
plt.ylabel("Tipos de rocha")
plt.title("Tipo de rocha dominante")
plt.show()

Quais são os regimes tectônicos?

In [0]:
volcanoes["Tectonic Setting"].value_counts().sort_values().plot.barh()
plt.ylabel("Regime tectônico")
plt.title("Regime tectônico")
plt.show()

## Distribuição espacial

In [0]:
fig = plt.figure(figsize=(12, 10))

m = Basemap(projection='mill', llcrnrlat=-80, urcrnrlat=80, llcrnrlon=-180, 
            urcrnrlon=180, lat_ts=20, resolution='c')

longitudes = np.array(volcanoes["Longitude"])
latitudes = np.array(volcanoes["Latitude"])

x, y = m(longitudes, latitudes)

m.plot(x, y, "o", markersize = 5, color = 'red', alpha=0.5, label='Localização')

# z = np.array(volcanoes['Elevation (Meters)'])
# m.scatter(x, y, z, alpha=0.2, label='Elevação (m)')

m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color='gray', lake_color='aqua')
m.drawmapboundary()
m.drawcountries()

plt.title("Erupções vulcânicas do Holoceno (vermelho)")

plt.show()

In [0]:
plt.close('all')

Como é a distribuição das erupções em relação à latitude? Há uma concentração maior na zona equatorial, no hemisfério norte ou no sul?

In [0]:
def division(data):
    
    # Split
    north_n = sum(data["Latitude"] >= 30)
    middle_n = sum(np.logical_and(data["Latitude"] < 30, data["Latitude"] > -30))
    south_n = sum(data["Latitude"] <= -30)
    
    # Sumário
    total = north_n + middle_n + south_n
    north_p = north_n / total * 100
    middle_p = middle_n / total * 100
    south_p = south_n / total * 100
    
    return [north_n, middle_n, south_n, north_p, middle_p, south_p]



In [0]:
# Segmentar dados por zona
volcanoes_split = division(volcanoes)

# Criar dataframe e inserir dados de frequência
volcanoes_split_df = pd.DataFrame(volcanoes_split[0:3],
                                  index=['Norte', 'Equatorial', 'Sul'],
                                  columns=['Frequência (n)'])

# Inserir porcentagens no dataframe
volcanoes_split_df['Porcentagem (%)'] = volcanoes_split[3:]

# Visualizar
volcanoes_split_df.iloc[::-1, 1].plot.barh();

Como é a distribuição a partir de 2012?

In [0]:
recent_active = volcanoes[(volcanoes["Last Known Eruption"] >= '2012 CE') & (volcanoes["Last Known Eruption"] <= '2016 CE')]

longitudes = recent_active["Longitude"].tolist()
latitudes = recent_active["Latitude"].tolist()

m = Basemap(projection='mill', 
            llcrnrlat=-80, 
            urcrnrlat=80, 
            llcrnrlon=-180, 
            urcrnrlon=180, lat_ts=20, resolution='c')

x, y = n(longitudes, latitudes)

plt.figure(figsize=(12, 10))
plt.title("Volcanoes (red) that were recently active and Earthquakes (green) in the last 5 years")

m.plot(x, y, "o", markersize = 5, color = 'red')
m.drawcoastlines()
m.fillcontinents(color='gray', lake_color='aqua')
m.drawmapboundary()
m.drawcountries()

plt.show()